In [1133]:
import cv2
import numpy as np

In [1134]:
first = True
def display(image, scale_x=0.5, scale_y=0.5):
    global first
    shrunk = cv2.resize(image, (0,0), fx=scale_x, fy=scale_y)
    cv2.imshow("Screen Location", shrunk)

    if cv2.waitKey(0 if first else 1000) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
    first = False

In [1135]:
billboard_colour = [150, 20, 20]
def strip_colours(image, preserve=billboard_colour):
    black_indices = np.where((image != preserve).all(axis=2))
    white_indices = np.where((image == preserve).all(axis=2))
    rtn = image.copy()
    rtn[black_indices] = [0,0,0]
    rtn[white_indices] = [255,255,255]
    return rtn

# noinspection PyDefaultArgument
def add_borders(image, border_color= [0,0,0], border_width=25):
    return cv2.copyMakeBorder(
        image,
        border_width,
        border_width,
        border_width,
        border_width,
        cv2.BORDER_CONSTANT,
        value=border_color
    )

def remove_borders(image, border_width=25):
  return image[border_width:-border_width, border_width:-border_width]

# noinspection PyShadowingNames
def get_largest_billboard_contour(image):
    edged_image = cv2.Canny(image, 30, 200)

    (contours, _) = cv2.findContours(
        edged_image.copy(),
        cv2.RETR_TREE,
        cv2.CHAIN_APPROX_SIMPLE
    )
    contours = sorted(
        contours,
        key=cv2.contourArea,
        reverse=True
    )[:10]


    best = (0,0)

    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        area = cv2.contourArea(contour)
        print("Found contour with {} nodes and total area {}"
              .format(len(approx), area))
        # if len(approx) == 4:
        if area > best[0]:
          best = (area, approx)

    return best[1] if best[0] > 0 else None

# noinspection PyShadowingNames
def get_contour_mask(image, contour):
    rtn = image.copy()
    cv2.drawContours(
        image=rtn,
        contours=[contour],
        contourIdx=-1,
        color=(255,255,255),
        thickness=cv2.FILLED
    )
    return rtn

def rotate(lst, x):
    return lst[-x:] + lst[:-x]
# noinspection PyShadowingNames
def get_homo_warped_decal(background, decal, contour):
    contour = [[x[0]] for x in contour]
    # contour = contour[
    contour = rotate(contour, 1)
    contour = np.float32(contour)

    h_decal, w_decal = decal.shape[:2]
    h_background, w_background = background.shape[:2]
    src = np.float32([
        [0, h_decal],
        [0,0],
        [w_decal, 0],
        [w_decal, h_decal],
    ])
    dst = np.float32(contour)
    h, mask = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

    flipped = np.flip(decal,axis=1)

    warped = cv2.warpPerspective(flipped, h, (w_background, h_background))
    return warped

In [1136]:
source_image = cv2.imread("../resources/new/015_00010.jpg")
labeled_image = cv2.imread("../resources/new/015_00010.png")

padded_source_image = add_borders(source_image)
padded_labeled_image = add_borders(labeled_image)

display(padded_source_image)
display(padded_labeled_image)

In [1137]:
binary_image = strip_colours(padded_labeled_image)
display(binary_image)

In [1138]:
contour = get_largest_billboard_contour(binary_image)

outlined = binary_image.copy()
outlined = cv2.drawContours(
    outlined,
    [contour],
    -1,
    [0,255,0],
    3
)
display(outlined)

Found contour with 4 nodes and total area 37642.5
Found contour with 4 nodes and total area 37641.5
Found contour with 4 nodes and total area 2413.5
Found contour with 4 nodes and total area 2406.5
Found contour with 8 nodes and total area 100.5
Found contour with 8 nodes and total area 100.0
Found contour with 12 nodes and total area 16.0
Found contour with 11 nodes and total area 13.0
Found contour with 6 nodes and total area 10.0
Found contour with 8 nodes and total area 10.0


In [1139]:
circles = padded_labeled_image.copy()
colors = [(0,255,0),(255,0,0),(255,255,0),(0,255,255)]
for i, center in enumerate([x[0] for x in contour]):
    cv2.circle(circles, tuple(center), 20, colors[i], -1)
display(circles)


In [1140]:
blank = np.zeros(padded_source_image.shape, dtype=np.uint8)
mask = get_contour_mask(blank, contour)
display(mask)
mask_inv = cv2.bitwise_not(mask)
display(mask_inv)

In [1141]:
labeled_image_no_billboard = cv2.bitwise_and(padded_labeled_image, mask_inv)
display(labeled_image_no_billboard)
source_image_no_billboard = cv2.bitwise_and(padded_source_image, mask_inv)
display(padded_source_image)
display(source_image_no_billboard)

In [1142]:
# decal_image = cv2.imread("../resources/new/decal_0.png")
decal_image = cv2.imread("../resources/new/milk_sip.png")
print(padded_source_image.shape)
print(decal_image.shape)
display(
    decal_image,
    scale_x=padded_source_image.shape[1]/decal_image.shape[1]*0.5,
    scale_y=padded_source_image.shape[0]/decal_image.shape[0]*0.5,
)

(1130, 1970, 3)
(585, 780, 3)


In [1143]:
warped = get_homo_warped_decal(outlined, decal_image, contour)
display(warped)

In [1144]:
output_image = cv2.bitwise_or(source_image_no_billboard, warped)
display(output_image)

In [1145]:
# final = cv2.bitwise_xor(display_image, display_image, mask=mask)
# display(final)